# Imports

In [1]:
import os
import numpy as np
import torch
import lightning as pl

from src.dataset.map_audio_dataset import MapAudioDataset
from src.datamodule.audio_data_module import AudioDataModule
from src.display.display_waveform import *

# Defentions

In [2]:
data_dir = os.path.join("..","..","data")

# Data

## Create DataModule Instance

In [3]:
datamodule = AudioDataModule(data_dir=data_dir,audio_dataset_class=MapAudioDataset)
datamodule.prepare_data()
datamodule.setup(stage='fit')
ref_batch_X,ref_batch_y = next(iter(datamodule.train_dataloader()))

100%|██████████| 2/2 [00:04<00:00,  2.39s/it]


## Display sample data

In [4]:
# sample_size = 6
# sample_index = np.random.randint(0, len(datamodule.train_dataset)-sample_size)
# for i in range(sample_index,sample_index+sample_size):
#     plot_waveforms(*datamodule.train_dataset[i])

# Net

In [5]:
from src.module.segan_generator import SEGAN_Generator
from src.module.segan_discriminator import SEGAN_Discriminator
from src.module.segan import SEGAN

generator = SEGAN_Generator()
discriminator = SEGAN_Discriminator(generator(ref_batch_X))
segan = SEGAN(generator,discriminator)

In [6]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar, DeviceStatsMonitor

# Callbacks
# checkpoint_callback = ModelCheckpoint(
#     monitor='val_loss',   # Metric to monitor
#     filename='segan-{epoch:02d}-{val_loss:.2f}',
#     save_top_k=3,         # Save the top 3 models
#     mode='min',           # Minimize the monitored metric (val_loss)
# )
# 
# early_stop_callback = EarlyStopping(
#     monitor='val_loss',
#     patience=10,          # Number of epochs with no improvement after which training will be stopped
#     verbose=True,
#     mode='min'
# )
progress_bar_callback = TQDMProgressBar()
device_stats_callback = DeviceStatsMonitor()
callbacks = []

In [7]:
# Initialize the Trainer with callbacks
torch.set_float32_matmul_precision("medium")
trainer = pl.Trainer(
    max_epochs=1,
    accelerator='gpu',
    log_every_n_steps=1,
    callbacks=callbacks,
    precision="16-mixed",
)

# Start training
trainer.fit(segan, datamodule=datamodule)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
  0%|          | 0/2 [00:00<?, ?it/s]

CONVERTING WAVS FROM ..\..\data\noisy_trainset_wav
CONVERTING WAVS FROM ..\..\data\clean_trainset_wav


 50%|█████     | 1/2 [00:05<00:05,  5.97s/it]

CONVERTING WAVS FROM ..\..\data\noisy_testset_wav
CONVERTING WAVS FROM ..\..\data\clean_testset_wav


100%|██████████| 2/2 [00:06<00:00,  3.19s/it]
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                | Params
------------------------------------------------------
0 | generator     | SEGAN_Generator     | 73.1 M
1 | discriminator | SEGAN_Discriminator | 24.4 M
2 | criterionGAN  | MSELoss             | 0     
3 | criterionL1   | L1Loss              | 0     
------------------------------------------------------
97.5 M    Trainable params
0         Non-trainable params
97.5 M    Total params
390.008   Total estimated model params size (MB)


...

Epoch 0:  99%|█████████▉| 164/166 [03:15<00:02,  0.84it/s, v_num=11]

NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


Epoch 0:  99%|█████████▉| 165/166 [03:16<00:01,  0.84it/s, v_num=11]

NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.
NaN or Inf found in input tensor.


AssertionError: x must have shape torch.Size([400, 2, 16384]) instead of torch.Size([33, 2, 16384])